# Modèle final (LSTM Bidirectionnal)

### Sommaire :
- Importation des packages
- Importation des données
- PRÉTRAITEMENT : VECTORISATION DU TEXTE
- CRÉATION DU DATASET TENSORFLOW
- CONSTRUCTION ET ENTRAÎNEMENT DU MODÈLE
- PRÉDICTIONS ET ÉVALUATIONS INITIALES
- CALCUL DES MÉTRIQUES AVEC KERAS
- CALCUL DES MÉTRIQUES AVEC SKLEARN
- SAUVEGARDE ET CHARGEMENT DU MODÈLE
- FONCTION DE SCORING D'UN COMMENTAIRE
- ÉVALUATION COMPLÉMENTAIRE AVEC ACCURACY_SCORE


# Importation des packages

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

# Import des modules Keras et sklearn
from tensorflow.keras.layers import TextVectorization, LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras import Sequential
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 1. Importation des données

Ajoutez un raccourci de ce dossier à votre google drive :

https://drive.google.com/drive/folders/1mx-CAzT10YKrmxHfYDP_1Oef7PVGUr7s?usp=sharing

In [4]:
# Importez le module de lecteur de Google.colab pour interagir avec Google Drive
#from google.colab import drive

# Montez Google Drive vers le répertoire '/Content/Drive'
#drive.mount('/content/drive', force_remount=True)

In [5]:
# Importation des données d'entraînement à partir de Google Drive
data = pd.read_csv('C:\\Users\\titou\\Downloads\\train.csv')

# Affichage rapide du dataframe pour vérifier les données importées
print(data)

# Affichage d'un exemple de commentaire toxique pour la classe 'identity_hate'
print(data.loc[data.identity_hate == 1].iloc[0].comment_text)

                      id                                       comment_text  \
0       0000997932d777bf  Explanation\nWhy the edits made under my usern...   
1       000103f0d9cfb60f  D'aww! He matches this background colour I'm s...   
2       000113f07ec002fd  Hey man, I'm really not trying to edit war. It...   
3       0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...   
4       0001d958c54c6e35  You, sir, are my hero. Any chance you remember...   
...                  ...                                                ...   
159566  ffe987279560d7ff  ":::::And for the second time of asking, when ...   
159567  ffea4adeee384e90  You should be ashamed of yourself \n\nThat is ...   
159568  ffee36eab5c267c9  Spitzer \n\nUmm, theres no actual article for ...   
159569  fff125370e4aaaf3  And it looks like it was actually you who put ...   
159570  fff46fc426af1f9a  "\nAnd ... I really don't think you understand...   

        toxic  severe_toxic  obscene  threat  insul

# 2. PRÉTRAITEMENT : VECTORISATION DU TEXTE

In [6]:
# Sélection des commentaires et des labels
x = data.comment_text  # Texte des commentaires
y = data[data.columns[2:]].values  # Labels des différentes classes de toxicité

# Définition du nombre maximum de caractéristiques (mots) à prendre en compte
max_words = 200000

# Initialisation du TextVectorization pour transformer le texte en séquences d'entiers
vectorizer = TextVectorization(max_tokens=max_words,
                               output_sequence_length=2000,
                               output_mode='int')

# Adapter le vectoriseur sur les données textuelles pour apprendre le vocabulaire
vectorizer.adapt(x.values)

# Affichage de la taille du vocabulaire appris
print("Taille du vocabulaire :", len(vectorizer.get_vocabulary()))

# Exemple de vectorisation d'un texte
print("Exemple de vectorisation :", vectorizer('Hello world, my name is Dualingo'))

# Conversion de tous les commentaires en séquences vectorisées
vectorized_text = vectorizer(x.values)

Taille du vocabulaire : 200000
Exemple de vectorisation : tf.Tensor([288 263  29 ...   0   0   0], shape=(2000,), dtype=int64)


# 3. CRÉATION DU DATASET TENSORFLOW

In [7]:
# Création d'un dataset TensorFlow à partir des données vectorisées et des labels
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
# Mise en cache du dataset pour améliorer les performances
dataset = dataset.cache()
# Mélange des données pour garantir que les batches sont aléatoires
dataset = dataset.shuffle(160000)
# Division des données en batches de taille 16
dataset = dataset.batch(16)
# Préchargement des données pour améliorer les performances d'entraînement
dataset = dataset.prefetch(8)

# Affichage d'un batch pour vérification
batch_x, batch_y = dataset.as_numpy_iterator().next()
print("Shape d'un batch (x):", batch_x.shape)
print("Shape d'un batch (y):", batch_y.shape)

# Division du dataset en ensembles d'entraînement, validation et test
total_batches = len(dataset)
train = dataset.take(int(total_batches * 0.7))  # 70% pour l'entraînement
val = dataset.skip(int(total_batches * 0.7)).take(int(total_batches * 0.2))  # 20% pour la validation
test = dataset.skip(int(total_batches * 0.9)).take(int(total_batches * 0.1))  # 10% pour le test

Shape d'un batch (x): (16, 2000)
Shape d'un batch (y): (16, 6)


# 4. CONSTRUCTION ET ENTRAÎNEMENT DU MODÈLE

In [8]:
# Création d'un modèle séquentiel
model = Sequential()
# Ajout d'une couche d'embedding pour convertir les indices de mots en vecteurs denses
model.add(Embedding(max_words + 1, 32))
# Ajout d'une couche LSTM bidirectionnelle pour capturer les dépendances dans les deux directions
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Ajout de couches denses avec activation ReLU pour l'apprentissage des caractéristiques complexes
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Ajout de la couche de sortie avec activation sigmoïde pour les prédictions binaires sur 6 classes
model.add(Dense(6, activation='sigmoid'))

# Affichage du résumé du modèle pour vérifier l'architecture
model.summary()

# Compilation du modèle avec une fonction de perte binaire et l'optimiseur Adam
model.compile(loss='BinaryCrossentropy', optimizer='Adam', metrics=['accuracy'])

# Entraînement du modèle sur les données d'entraînement avec validation sur les données de validation
history = model.fit(train, epochs=20, validation_data=val)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 5001s 715ms/step - accuracy: 0.9690 - loss: 0.0822 - val_accuracy: 0.9939 - val_loss: 0.0473
Epoch 2/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 33328s 5s/step - accuracy: 0.9940 - loss: 0.0457 - val_accuracy: 0.9941 - val_loss: 0.0409
Epoch 3/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 5691s 815ms/step - accuracy: 0.9941 - loss: 0.0405 - val_accuracy: 0.9942 - val_loss: 0.0349
Epoch 4/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 7527s 1s/step - accuracy: 0.9937 - loss: 0.0351 - val_accuracy: 0.9946 - val_loss: 0.0317
Epoch 5/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 32836s 5s/step - accuracy: 0.9908 - loss: 0.0317 - val_accuracy: 0.9940 - val_loss: 0.0275
Epoch 6/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 19084s 3s/step - accuracy: 0.9935 - loss: 0.0280 - val_accuracy: 0.9931 - val_loss: 0.0244
Epoch 7/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 4830s 692ms/step - accuracy: 0.9927 - loss: 0.0255 - val_accuracy: 0.9938 - val_loss: 0.0221
Epoch 8/20
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 4907s 703ms/step - acc

# 5. PRÉDICTIONS ET ÉVALUATIONS INITIALES

In [9]:
# Prédiction sur un exemple de texte
input_text = vectorizer('You are so dumbt! if i see you, you are dead')
print("Prédiction (exemple 1) :", model.predict(np.array([input_text])))
print("Prédiction (exemple 2) :", model.predict(np.expand_dims(input_text, 0)))

# Evaluation sur un batch du test set
batch_x, batch_y = test.as_numpy_iterator().next()
print("Prédictions sur batch test :", (model.predict(batch_x) > 0.5).astype(int))
print("Labels réels :", batch_y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 613ms/step
Prédiction (exemple 1) : [[0.8467318  0.03478251 0.03738671 0.24974585 0.36224842 0.00687728]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Prédiction (exemple 2) : [[0.8467318  0.03478251 0.03738671 0.24974585 0.36224842 0.00687728]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 665ms/step
Prédictions sur batch test : [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Labels réels : [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


# 6. CALCUL DES MÉTRIQUES AVEC KERAS

In [10]:
# Initialisation des métriques de précision et de rappel
precision = Precision()
recall = Recall()

# Boucle sur chaque batch du jeu de données de test
for batch in test.as_numpy_iterator():
    batch_x, batch_y = batch
    # Prédiction des labels pour le batch courant
    yhat = model.predict(batch_x)
    # Aplatir les tableaux pour le calcul des métriques
    precision.update_state(batch_y.flatten(), yhat.flatten())
    recall.update_state(batch_y.flatten(), yhat.flatten())

# Calcul des résultats finaux pour la précision et le rappel
precision = precision.result().numpy()
recall = recall.result().numpy()
# Calcul du F1-score à partir de la précision et du rappel
f1score = (2 * precision * recall) / (precision + recall)

# Affichage des résultats
print(f"Precision: {precision}\nRecall: {recall}\nF1-score: {f1score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

# 7. CALCUL DES MÉTRIQUES AVEC SKLEARN

In [11]:
# Initialisation des listes pour stocker les vraies étiquettes et les prédictions
y_true = []
y_pred = []

# Boucle sur chaque batch du jeu de données de test
for batch in test.as_numpy_iterator():
    batch_x, batch_y = batch
    # Prédiction des labels pour le batch courant
    yhat = model.predict(batch_x)
    # Ajout des vraies étiquettes et des prédictions aux listes
    y_true.append(batch_y)
    y_pred.append(yhat)

# Conversion des listes en tableaux numpy
y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

# Seuil de 0.5 pour binariser les prédictions
y_pred_bin = (y_pred > 0.5).astype(int)

# Calcul des métriques par label
precision_per_label = precision_score(y_true, y_pred_bin, average=None)
recall_per_label = recall_score(y_true, y_pred_bin, average=None)
f1_per_label = f1_score(y_true, y_pred_bin, average=None)

# Affichage des résultats par label
print(f"Precision par label: {precision_per_label}")
print(f"Recall par label: {recall_per_label}")
print(f"F1-score par label: {f1_per_label}")

# Calcul des F1-scores globaux
f1_macro = f1_score(y_true, y_pred_bin, average="macro")
f1_micro = f1_score(y_true, y_pred_bin, average="micro")

# Affichage des F1-scores globaux
print(f"F1-score macro (moyenne): {f1_macro}")
print(f"F1-score micro (global): {f1_micro}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

# 8. SAUVEGARDE ET CHARGEMENT DU MODÈLE

In [12]:
# sauvegarde du modele
model.save('Final_LSTM.h5')
# chargement du modele
model = tf.keras.models.load_model('Final_LSTM.h5')
print("Modèle chargé :", model)

Modèle chargé : <Sequential name=sequential, built=True>


# 9. FONCTION DE SCORING D'UN COMMENTAIRE

In [13]:
def score_comment(comment):
    # Vectorisation du commentaire pour le transformer en séquence d'entiers
    vectorized_comment = vectorizer([comment])
    
    # Prédiction des probabilités de toxicité pour chaque classe
    results = model.predict(vectorized_comment)
    
    # Initialisation d'une chaîne de caractères pour stocker les résultats
    text = ""
    
    # Boucle sur chaque classe de toxicité pour formater les résultats
    for idx, col in enumerate(data.columns[2:]):
        # Ajout du nom de la classe et du résultat de la prédiction (True si > 0.5, sinon False)
        text += f'{col}: {results[0][idx] > 0.5}\n'
    
    # Retourne le texte formaté avec les résultats
    return text

# Exemple d'utilisation de la fonction avec un commentaire toxique
print(score_comment("FUCK YOU"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step
toxic: True
severe_toxic: True
obscene: True
threat: False
insult: True
identity_hate: False



# 10. ÉVALUATION COMPLÉMENTAIRE AVEC ACCURACY_SCORE

In [14]:
# bout de code pour pouvoir évaluer des phrases seuls, suffit de changer txt
txt = "I hate you"
vec_txt = vectorizer(txt)
print("Prédiction pour la phrase mise est :", model.predict(np.expand_dims(vec_txt, 0)))
print("Labels :", data.columns[2:])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Prédiction pour la phrase mise est : [[0.7937336  0.00762909 0.03614698 0.00867366 0.23291218 0.00215738]]
Labels : Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')


In [15]:
# Préparation des données de test pour évaluer l'accuracy
x_test = np.expand_dims(vec_txt.numpy(), 0)
y_test = [[1, 0, 0, 0, 0, 0]]
for batch in test.as_numpy_iterator():
    batch_x, batch_y = batch
    x_test = np.concatenate((x_test, batch_x))
    y_test = np.concatenate((y_test, batch_y))
yhat = model.predict(x_test)
yhat = (yhat > 0.5).astype(int)
print("Accuracy :", accuracy_score(np.array(y_test).flatten(), yhat.flatten()))

499/499 ━━━━━━━━━━━━━━━━━━━━ 80s 160ms/step
Accuracy : 0.9975030819699534
